In [1]:
from utils import *

hete = Heterogeneity()

In [2]:
facies = hete.load_facies()

hete_fluv, hete_gaus = hete.load_perm_poro()

Facies shape: (1000, 256, 256)
Permf: (1000, 65536) | Permg: (1000, 65536)
Fluvial heterogeneity: (1000, 256, 256, 2) | Gaussian heterogeneity: (1000, 256, 256, 2)


In [27]:
porog = 10**((hete.permg-7)/10)
porof = 10**((hete.permf-7)/10)

g, f = {}, {}
for i in range(1000):
    pd.DataFrame(porog[i].reshape(256,256)).to_csv('data/poro_gaus/poroG{}.csv'.format(i))
    pd.DataFrame(porof[i].reshape(256,256)).to_csv('data/poro_fluv/poroF{}.csv'.format(i))
    pd.DataFrame(hete.permg[i].reshape(256,256)).to_csv('data/perm_gaus/logpermG{}.csv'.format(i))
    pd.DataFrame(hete.permf[i].reshape(256,256)).to_csv('data/perm_fluv/logpermF{}.csv'.format(i))

In [39]:
hete.hete_fluv.shape

(1000, 256, 256, 2)

In [40]:
for i in range(1000):
    np.save('data/hete_gaus/gaussian{}.npy'.format(i), hete.hete_gaus[i])
    np.save('data/hete_fluv/fluvial{}.npy'.format(i), hete.hete_fluv[i])

In [ ]:
fig, axs = plt.subplots(3, 10, figsize=(20,5))
im0, im1, im2 = {}, {}, {}
for j in range(10):
    k = j*75
    im0 = axs[0,j].imshow(facies[k], 'jet')
    im1 = axs[1,j].imshow(hete_fluv[k,:,:,0], 'jet')
    im2 = axs[2,j].imshow(hete_fluv[k,:,:,1], 'jet')
    for i in range(3):
        axs[i,j].set(xticks=[], yticks=[])
plt.colorbar(im0, fraction=0.046, pad=0.04)
plt.colorbar(im1, fraction=0.046, pad=0.04)
plt.colorbar(im2, fraction=0.046, pad=0.04)

plt.show()